# Coursera Capstone Project

## Battle of the Neighborhoods



### Introduction:

A restauranteer is seeking to open a new restaurant.  The restauranteer would like to open a new location in the Charlotte area but doesn't know what area to invest in.  He hopes to open in an area that currently lacks restaurant venues and is in an area easily accessible (near public transportation) for their future customers.

Charlotte is the most populous metropolitan area in North Carolina, and 15th most populous in the United States.  The population here is growing rapidly, with the US Census Bureau ranking it as the top area for millennial population growth from 2005 to 2015.  Millennials are always looking for that "next trendy restaraunt" in neighborhoods they may be unfamiliar with.  In addition, millennials are less likely to own personal transportation as compared to their predecessors. Together, these facts make targetting an area for a restaurant that is easy for millennials to travel to via public transportation important.

### Data:

The new location's area will be chosen by the results of a k-means cluster analysis of Charlotte's Zip Codes ([Link to Zip Codes online](https://www.zipcodestogo.com/North%20Carolina/).  These zip codes will be clustered on the basis of top venues in each area obtained from the Foursquare API.  Geographical locations of the neighborhood will be retrieved with the function geocoder from the geopy package in python.  The cluster analysis will be completed with the KMeans function from the sklearn package.



In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from bs4 import BeautifulSoup

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


In [41]:
table = pd.read_html('https://www.zipcodestogo.com/North%20Carolina/')[1]
table.columns = table.iloc[1]
table = table[2:]
table = table.drop(['Zip Code Map', 'County'], axis=1).reset_index(drop=True)
print(table)

1    Zip Code            City
0       27006         Advance
1       27007          Ararat
2       27009    Belews Creek
3       27010        Bethania
4       27011       Boonville
5       27012        Clemmons
6       27013       Cleveland
7       27014       Cooleemee
8       27016         Danbury
9       27017          Dobson
10      27018       East Bend
11      27019       Germanton
12      27020    Hamptonville
13      27021            King
14      27022     Lawsonville
15      27023      Lewisville
16      27024          Lowgap
17      27025         Madison
18      27027         Mayodan
19      27028      Mocksville
20      27030      Mount Airy
21      27031    White Plains
22      27040       Pfafftown
23      27041  Pilot Mountain
24      27042       Pine Hall
25      27043        Pinnacle
26      27045      Rural Hall
27      27046     Sandy Ridge
28      27047          Siloam
29      27048      Stoneville
...       ...             ...
1061    28782           Tryon
1062    28